引入numpy并且加载数据，看看数据是什么样子

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.datasets import load_boston
boston = load_boston()

分别定义几个函数，用于编写迭代算法。  
1. 示性函数 indicate function  
2. 用最大元素替换向量值  
3. 利用如下计算公式计算梯度：
$$ f(x) = \frac{1}{2}\parallel X^\mathrm{ T }\beta-y \parallel^2 $$  
$$ grad = 2X\mathrm(X^\mathrm{ T }\beta-y) $$  
而后，则根据PDF的公式，编写迭代函数算lasso。  
其实我发现PPT有点问题，PPT说调参调整的是lambda，但是发现调整的好像是算式里面的mu，因为算式里面的mu才是控制每一步DS长度的参数，而且如果把mu设置成1，调整lambda，算法会不收敛。

In [ ]:
def sign(x):
    for i in range(len(np.copy(x))):
        if x[i,0] != 0:
            res = x[i,0] / abs(x[i,0])
        else:
            res = 0;
    return res

def vec_max(MAX_VALUE, v):
    for i in range(len(v)):
        v[i] = max(MAX_VALUE, v[i,0])
    return v

def grf(A,x,y):
    return A.T.dot(A.dot(x) - y)

In [ ]:
def lasso_proximal(X,y,lambda_set,MAX_ITER=3000,epsilon = 1e-10):
    n = X.shape[1]
    beta = np.zeros((n,1))
    mu = np.linspace(1,1,n).reshape(1,n)
    y = y.reshape(len(y),1)
    for i in range(MAX_ITER):
        beta_k1 = beta - lambda_set*grf(X,beta,y)
        beta_k1 = vec_max(0, abs(beta_k1)-mu.T * lambda_set) * sign(beta_k1)
        print(sum(abs(beta_k1 - beta)))
        if sum(abs(beta_k1 - beta)) <= epsilon:
            beta = beta_k1
            break
        else:
            beta = beta_k1
    return beta

定义函数标准化数据，划分训练集测试集，然后初步拟定超参数，跑一下算法，看一下MSE，对比下

In [ ]:
def normalization_X(x):
    return (x-x.mean(axis=0))/x.std(axis=0)

def normalization_y(y): 
    return (y-y.mean)/y.std  

In [ ]:
X = boston.data
y = boston.target
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=13)
X_train_n = normalization_X(X_train)
X_test_n = normalization_X(X_test)
y_train_n = normalization_X(y_train)
y_test_n = normalization_X(y_test)

接下来自己编写交叉验证，来找到MSE最小的k，也就是学习率，最小用五折的平均MSE找到

In [ ]:
num_folds = 5
lambda_choices = [pow(10, -3-0.1*i) for i in range(6)]
X_train_folds = []
y_train_folds = []
k_to_accuracies = {}

X_train_folds = np.array_split(X_train_n, num_folds)
y_train_folds = np.array_split(y_train_n, num_folds)

for k in lambda_choices:
    mse = []
    print("现在的学习率是："+str(k))
    for i in range(num_folds):
        Xtr = np.concatenate(X_train_folds[:i] + X_train_folds[i+1:])
        ytr = np.concatenate(y_train_folds[:i] + y_train_folds[i+1:])
        Xcv = X_train_folds[i]
        ycv = y_train_folds[i]
        beta = lasso_proximal(Xtr,ytr,k,MAX_ITER=100000)
        y_train_predict=Xcv.dot(beta)
        mse_predict = mean_squared_error(ycv,y_train_predict)
        mse.append(mse_predict)
    k_to_accuracies[k] = mse

for k in sorted(k_to_accuracies):
    avg_mse = sum(k_to_accuracies[k][:])/num_folds
    print('k='+str(k)+'   '+'average_mse='+str(avg_mse)+'\n\t')

可以看到，当k=2.5118864315095823e-07   average_mse=0.3641587835977935达到最小。  
最后直接用训练集按照找到的最优的学习率来训练一下，然后看看参数beta的情况。

In [ ]:
beta_final = lasso_proximal(X_train_n,y_train_n,2.5118864315095823e-07,MAX_ITER=100000)
y_train_predict = X_train_n.dot(beta_final)
mse_final = mean_squared_error(y_train_n,y_train_predict)

In [ ]:
mse_final

可以看到，最后的MSE大概是0.3534，那么接下来看一下训练出来的参数，哪些变量显著哪些不显著。

In [ ]:
boston['feature_names_Chinese'] = np.array(['城市人均犯罪率',
                                            '占地面积超过 25,000 平方英尺的住宅用地比例',
                                            '每个城镇非零售业务的比例',
                                            'Charles River 虚拟变量（如果是河道，则为 1; 否则为 0） ',
                                            '一氧化氮浓度（每千万份） ',
                                            '每间住宅的平均房间数',
                                            '1940 年以前建造的自住单位比例 ',
                                            '加权距离波士顿的五个就业中心',
                                            '径向高速公路的可达性指数 ', 
                                            '每 10,000 美元的全额物业税率',
                                            '城镇的学生与教师比例 ',
                                            'B=1000(Bk −0.63)2 其中 Bk 是城镇黑人的比例 ',
                                            '人口状况下降 '])
for i in range(len(beta_final)):
    print(beta_final[i], boston['feature_names'][i], boston['feature_names_Chinese'][i])
    if abs(beta_final[i]) <= 1e-5:
        print('   ' + '此变量不显著')

事实上，被lasso惩罚到0的系数可以认为是对回归帮助不大的系数，即对房价影响不显著。根据上述结果，我们可以看到：大住宅用地比例、有无河道、住宅房间数、1940年以前建造的自主单位比例、径向高速公路的可达性指数、美元的全额物业税率、城镇黑人的比例是不显著的，其余变量是显著的。这说明，这些变量与放假并没有很强的相关关系。  